# Transportation Problem: Example

In [1]:
from pyomo.environ import *
import numpy as np

We need to transport mangoes from a number of **Farms** to a number of **Markets**. Each **Farm** $i$ has a fixed *supply* of mangoes ($a_i$ in kg) and each **Market** $j$ has a fixed *demand* for mangoes ($b_i$ in kg). The distance from every **Farm** $i$ to every **Market** $j$ is given by $d_{ij}$ (in km) and the transportation cost is given as 1200.00 PhP/kg/km.

Let $x_{ij}$ be the amount of mangoes (kg) to be transported from **Farm** $i$ to **Market** $j$. Treat the amounts $x_{ij}$ as continuous variables. Your task is to decide all the values of $x_{ij}$ so that the *total transportation cost* is minimum:

$$ \min{\sum_i \sum_j \left( 1200 \frac{\mathrm{PhP}}{\mathrm{kg}\cdot \mathrm{km}}\times x_{ij} \mathrm{kg} \times d_{ij} \mathrm{km}\right)} $$

while satisfying the following constraints:
* Each market demand $j$ must be met:    $ \sum_i x_{ij} \geq b_j$  for $j=1, 2, 3, 4$
* Each supply $i$ must not be exceeded:  $ \sum_j x_{ij} \leq a_i$  for $i=1, 2, 3$
* All $x_{ij}$ should be non-negative:   $ x_{ij} \geq 0$ for all $i, j$

Solve this problem given the following information:

| Distance $d_{ij}$ | Market 1 | Market 2 | Market 3 | Market 4 | Supply $a_i$ |
| ----------------- | -------- | -------- | -------- | -------- | ------------ |
| **Farm 1**        | 2.5      | 3.8      | 1.8      | 2.0      | 350          |
| **Farm 2**        | 2.0      | 3.1      | 2.5      | 1.2      | 250          |
| **Farm 3**        | 3.2      | 4.2      | 2.2      | 3.5      | 500          |
| **Demand** $b_j$  | 250      | 400      | 125      | 225      |              |

Report all answers ($x_{ij}$) accurate to 2 decimal places. Also, report the minimum transportation cost.

In [2]:
# Create the model object

def create_Transpo_mdl(Src, Tgt, d, Sup, Dem):
    # Src  = list of sources (farms)
    # Tgt  = list of targets (markets)
    # d    = distance matrix
    # Sup  = set of supply values
    # Dem  = set of demand values
    
    model = ConcreteModel(name="Transpo1")

    # Declare decision variables and parameters
    model.x = Var(Src, Tgt, domain=NonNegativeReals)
    model.d = Param(Src, Tgt, initialize=lambda model, i, j: d[i-1][j-1])
    
    def obj_rule(mdl):
        return sum(1200*mdl.x[i,j]*mdl.d[i,j] for i in Src for j in Tgt)
    
    def demand_rule(mdl, j):
        return sum(mdl.x[i,j] for i in Src) >= Dem[j-1]
    
    def supply_rule(mdl, i):
        return sum(mdl.x[i,j] for j in Tgt) <= Sup[i-1] 
    
    model.obj = Objective(rule=obj_rule)
    model.dem = Constraint(Tgt, rule=demand_rule)
    model.sup = Constraint(Src, rule=supply_rule)
    
    return model

# Specify the data
Src = RangeSet(3)
Tgt = RangeSet(4)
dist = [[2.5, 3.8, 1.8, 2.0],
        [2.0, 3.1, 2.5, 1.2],
        [3.2, 4.2, 2.2, 3.5]]
Sup = [350, 250, 500]
Dem = [250, 400, 125, 225]

# Solve the model
model = create_Transpo_mdl(Src, Tgt, dist, Sup, Dem)
solver = SolverFactory("glpk") 
res = solver.solve(model)
model.display() 
model.x.pprint()
res.write()

Model Transpo1

  Variables:
    x : Size=12, Index=[1:3]*[1:4]
        Key    : Lower : Value : Upper : Fixed : Stale : Domain
        (1, 1) :     0 : 250.0 :  None : False : False : NonNegativeReals
        (1, 2) :     0 : 100.0 :  None : False : False : NonNegativeReals
        (1, 3) :     0 :   0.0 :  None : False : False : NonNegativeReals
        (1, 4) :     0 :   0.0 :  None : False : False : NonNegativeReals
        (2, 1) :     0 :   0.0 :  None : False : False : NonNegativeReals
        (2, 2) :     0 :  25.0 :  None : False : False : NonNegativeReals
        (2, 3) :     0 :   0.0 :  None : False : False : NonNegativeReals
        (2, 4) :     0 : 225.0 :  None : False : False : NonNegativeReals
        (3, 1) :     0 :   0.0 :  None : False : False : NonNegativeReals
        (3, 2) :     0 : 275.0 :  None : False : False : NonNegativeReals
        (3, 3) :     0 : 125.0 :  None : False : False : NonNegativeReals
        (3, 4) :     0 :   0.0 :  None : False : False : N

Based on the solution, it is optimal that no mangoes should be transported to the nearest market to Farm 1 (i.e. Market 3, which is only 1.8 km away). Solve the problem again but with a constraint that at least 150 kg should be transported from Farm 1 to Market 3, then prove that this is a sub-optimal solution.

In [3]:
def create_Transpo_mdl2(Src, Tgt, d, Sup, Dem):
    # Src  = list of sources (farms)
    # Tgt  = list of targets (markets)
    # d    = distance matrix
    # Sup  = set of supply values
    # Dem  = set of demand values
    
    model = ConcreteModel(name="Transpo1")

    # Declare decision variables and parameters
    model.x = Var(Src, Tgt, domain=NonNegativeReals)
    model.d = Param(Src, Tgt, initialize=lambda model, i, j: d[i-1][j-1])
    
    def obj_rule(mdl):
        return sum(1200*mdl.x[i,j]*mdl.d[i,j] for i in Src for j in Tgt)
    
    def demand_rule(mdl, j):
        return sum(mdl.x[i,j] for i in Src) >= Dem[j-1]
    
    def supply_rule(mdl, i):
        return sum(mdl.x[i,j] for j in Tgt) <= Sup[i-1] 
    
    model.obj = Objective(rule=obj_rule)
    model.dem = Constraint(Tgt, rule=demand_rule)
    model.sup = Constraint(Src, rule=supply_rule)
    model.con = Constraint(expr=model.x[1,3] >= 150)
    
    return model

# Solve the model
model = create_Transpo_mdl2(Src, Tgt, dist, Sup, Dem)
solver = SolverFactory("glpk") 
res = solver.solve(model)
model.display() 
model.x.pprint()
res.write()

Model Transpo1

  Variables:
    x : Size=12, Index=[1:3]*[1:4]
        Key    : Lower : Value : Upper : Fixed : Stale : Domain
        (1, 1) :     0 : 200.0 :  None : False : False : NonNegativeReals
        (1, 2) :     0 :   0.0 :  None : False : False : NonNegativeReals
        (1, 3) :     0 : 150.0 :  None : False : False : NonNegativeReals
        (1, 4) :     0 :   0.0 :  None : False : False : NonNegativeReals
        (2, 1) :     0 :  25.0 :  None : False : False : NonNegativeReals
        (2, 2) :     0 :   0.0 :  None : False : False : NonNegativeReals
        (2, 3) :     0 :   0.0 :  None : False : False : NonNegativeReals
        (2, 4) :     0 : 225.0 :  None : False : False : NonNegativeReals
        (3, 1) :     0 :  25.0 :  None : False : False : NonNegativeReals
        (3, 2) :     0 : 400.0 :  None : False : False : NonNegativeReals
        (3, 3) :     0 :   0.0 :  None : False : False : NonNegativeReals
        (3, 4) :     0 :   0.0 :  None : False : False : N